In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

## Radiation model using the commuters data

In [2]:
radiation_municipality_path = "./results/radiation_model_number_of_commuters_as_prior.csv"
commuters_static_radiation_municipality = {}
with open(radiation_municipality_path, 'r') as data:
    line_counter = 0
    for line in data:
        line_counter += 1
        if line_counter > 1:
            fields = line.strip().split(",")
            origin = fields[0]
            destination = fields[1]
            flow = float(fields[-1])
            commuters_static_radiation_municipality[(origin, destination)] = flow

In [3]:
len(commuters_static_radiation_municipality)

95790

# Statis Gravity model in the level of Municipalities

In [5]:
gravity_municipality_path = "./results/gravity_model_exp_avg_surface_with_commuter.csv"
commuters_static_gravity_municipality = {}
with open(gravity_municipality_path, 'r') as data:
    line_counter = 0
    for line in data:
        line_counter += 1
        if line_counter > 1:
            fields = line.strip().split(",")
            origin = fields[0]
            destination = fields[1]
            flow = float(fields[2])
            commuters_static_gravity_municipality[(origin, destination)] = flow

In the results for the gravity model, those municipality pairs for which zero flow has been predicted are missing. To make it similar to the results for the radiation model, we manually add them. 

In [6]:
len(commuters_static_gravity_municipality)

96410

In [7]:
for city_pair in commuters_static_radiation_municipality.keys():
    if city_pair not in commuters_static_gravity_municipality.keys():
        commuters_static_gravity_municipality[city_pair] = 0

In [8]:
len(commuters_static_gravity_municipality)

96410

## make the estimations dynamic

In [9]:
def map_municiplaities_to_facebook_districts(data_path):
    line_counter = 0
    m_to_f = {}
    f_to_m = {}
    with open(data_path, 'r') as data:
        for line in data:
            line_counter += 1
            ### one header line
            if line_counter > 1:
                fields = line.strip().split(',')
                municipality = fields[0]
                facebook_region = fields[1]
                m_to_f[municipality] = facebook_region
                if facebook_region not in f_to_m.keys():
                    f_to_m[facebook_region] = set()
                f_to_m[facebook_region].add(municipality)
    return f_to_m, m_to_f

In [10]:
municipality_to_facebook_path = "./basic_data/commuters_data/finland/mapping_municipality_to_facebook_regions_finland.csv"
f_to_m, m_to_f = map_municiplaities_to_facebook_districts(municipality_to_facebook_path)

In [11]:
f_to_m

{'Western Finland': {'Akaa',
  'Alajärvi',
  'Alavus',
  'Asikkala',
  'Aura',
  'Eura',
  'Eurajoki',
  'Evijärvi',
  'Forssa',
  'Halsua',
  'Hankasalmi',
  'Harjavalta',
  'Hattula',
  'Hausjärvi',
  'Hollola',
  'Honkajoki',
  'Huittinen',
  'Humppila',
  'Hämeenkyrö',
  'Hämeenlinna',
  'Ikaalinen',
  'Ilmajoki',
  'Isojoki',
  'Isokyrö',
  'Jakobstad',
  'Janakkala',
  'Jokioinen',
  'Joutsa',
  'Juupajoki',
  'Jyväskylä',
  'Jämijärvi',
  'Jämsä',
  'Kaarina',
  'Kangasala',
  'Kankaanpää',
  'Kannonkoski',
  'Kannus',
  'Karijoki',
  'Karstula',
  'Karvia',
  'Kaskinen',
  'Kauhajoki',
  'Kauhava',
  'Kaustinen',
  'Keuruu',
  'Kihniö',
  'Kimitoön',
  'Kinnula',
  'Kivijärvi',
  'Kokemäki',
  'Kokkola',
  'Konnevesi',
  'Korsholm',
  'Korsnäs',
  'Koski_Tl',
  'Kristinestad',
  'Kronoby',
  'Kuhmoinen',
  'Kuortane',
  'Kurikka',
  'Kustavi',
  'Kyyjärvi',
  'Kärkölä',
  'Lahti',
  'Laihia',
  'Laitila',
  'Lappajärvi',
  'Lapua',
  'Larsmo',
  'Laukaa',
  'Lempäälä',
  'Lesti

In [12]:
def map_municiplaities_to_google_districts(data_path):
    line_counter = 0
    m_to_g = {}
    g_to_m = {}
    with open(data_path, 'r') as data:
        for line in data:
            line_counter += 1
            ### one header line
            if line_counter > 1:
                fields = line.strip().split(',')
                if len(fields) == 5:
                    municipality = fields[1]
                    google_region = fields[4]
                    m_to_g[municipality] = google_region
                    if google_region not in g_to_m.keys():
                        g_to_m[google_region] = set()
                    g_to_m[google_region].add(municipality)
    return g_to_m, m_to_g

In [13]:
municipality_to_google_path = "./basic_data/commuters_data/finland/mapping_municipality_to_google_regions_finland.csv"
g_to_m, m_to_g = map_municiplaities_to_google_districts(municipality_to_google_path)

In [14]:
g_to_m.keys()

dict_keys(['Southern Ostrobothnia', 'Northern Ostrobothnia', 'Päijänne Tavastia', 'Uusimaa', 'Southwest Finland', 'Pirkanmaa', '', 'Lapland', 'Satakunta', 'Tavastia Proper', 'Central Ostrobothnia', 'Kymenlaakso', 'Central Finland', 'North Karelia', 'Southern Savonia', 'Kainuu', 'Northern Savonia', 'South Karelia', 'Ostrobothnia'])

This function will read the file `kunta_utf-8_trimmed_include_both_swedish_and_Finnish.csv` and generate 2 dictionaries. The first one will map the hospital care districts (HCD) to the municipalities. The second one will do the inverse order.

In [15]:
m_to_h_data_converted_format = "./data/kunta_utf-8_trimmed_include_both_swedish_and_Finnish.csv"

In [16]:
def map_municiplaities_to_hospital_district_areas(data_path):
    line_counter = 0
    m_to_h = {}
    h_to_m = {}
    with open(data_path, 'r') as data:
        for line in data:
            line_counter += 1
            ### the enteries start from 6th line
            if line_counter > 5:
                fields = line.strip().split(';')
                municipality_string = fields[1]
                #print(municipality_string)
                municipality = municipality_string[1:-1]
                #print(municipality)
                hospital_string = fields[3]
                hospital_district = hospital_string[1:-1]
                #print(hospital_district)
                m_to_h[municipality] = hospital_district
                if hospital_district not in h_to_m.keys():
                    h_to_m[hospital_district] = set()
                h_to_m[hospital_district].add(municipality)
    return h_to_m, m_to_h

Storing the results of running the function

In [17]:
h_to_m, m_to_h = map_municiplaities_to_hospital_district_areas(m_to_h_data_converted_format)
# print(h_to_m)
# print(m_to_h)

In [18]:
h_to_m

{'Pirkanmaa Hospital District': {'Akaa',
  'Hämeenkyrö',
  'Ikaalinen',
  'Juupajoki',
  'Jämsä',
  'Kangasala',
  'Kihniö',
  'Kuhmoinen',
  'Lempäälä',
  'Mänttä-Vilppula',
  'Nokia',
  'Orivesi',
  'Parkano',
  'Pirkkala',
  'Pälkäne',
  'Ruovesi',
  'Sastamala',
  'Tampere',
  'Urjala',
  'Valkeakoski',
  'Vesilahti',
  'Virrat',
  'Ylöjärvi'},
 'South Ostrobothnia Hospital District': {'Alajärvi',
  'Alavus',
  'Evijärvi',
  'Ilmajoki',
  'Isojoki',
  'Isokyrö',
  'Karijoki',
  'Kauhajoki',
  'Kauhava',
  'Kuortane',
  'Kurikka',
  'Lappajärvi',
  'Lapua',
  'Seinäjoki',
  'Soini',
  'Teuva',
  'Vimpeli',
  'Ähtäri'},
 'North Ostrobothnia Hospital District': {'Alavieska',
  'Haapajärvi',
  'Haapavesi',
  'Hailuoto',
  'Ii',
  'Kalajoki',
  'Kempele',
  'Kuusamo',
  'Kärsämäki',
  'Liminka',
  'Lumijoki',
  'Merijärvi',
  'Muhos',
  'Nivala',
  'Oulainen',
  'Oulu',
  'Pudasjärvi',
  'Pyhäjoki',
  'Pyhäjärvi',
  'Pyhäntä',
  'Raahe',
  'Sievi',
  'Siikajoki',
  'Siikalatva',
  'Taiv

# read faceook mutipliers

In [19]:
# filename = '/home/huangz4/nordicmathcovid-mobility/public_mobility_data/Facebook/movement-range-data-2020-03-01--2020-12-31.txt'
# df = pd.read_csv(filename, sep='\t', parse_dates=['ds'])

# df_facebook_fin = df[df['country'] == 'FIN'].copy(True)

# df_facebook_fin.to_csv('./public_mobility_data/facebook_mutiplier.csv', index=False)

In [21]:
df_facebook_fin = pd.read_csv('./public_mobility_data/facebook_mutiplier.csv', 
                              parse_dates=['ds'])

In [22]:
facebook_multiplier_book = dict(zip(zip(df_facebook_fin.ds, df_facebook_fin.polygon_name), df_facebook_fin.all_day_bing_tiles_visited_relative_change))
facebook_multiplier_book = {(key[0].to_pydatetime(), key[1]):value for (key, value) in facebook_multiplier_book.items()}
facebook_multiplier_book = {(key[1], key[0].day, key[0].month, key[0].year):value for (key, value) in facebook_multiplier_book.items()}

In [23]:
facebook_multiplier_book

{('Eastern Finland', 1, 3, 2020): -0.04944,
 ('Eastern Finland', 2, 3, 2020): 0.04129,
 ('Eastern Finland', 3, 3, 2020): -0.016659999999999998,
 ('Eastern Finland', 4, 3, 2020): 0.00115,
 ('Eastern Finland', 5, 3, 2020): -0.05467,
 ('Eastern Finland', 6, 3, 2020): -0.05869,
 ('Eastern Finland', 7, 3, 2020): -0.00229,
 ('Eastern Finland', 8, 3, 2020): 0.01941,
 ('Eastern Finland', 9, 3, 2020): 0.01904,
 ('Eastern Finland', 10, 3, 2020): -0.0235,
 ('Eastern Finland', 11, 3, 2020): 0.0032600000000000003,
 ('Eastern Finland', 12, 3, 2020): -0.02051,
 ('Eastern Finland', 13, 3, 2020): -0.0564,
 ('Eastern Finland', 14, 3, 2020): -0.10319,
 ('Eastern Finland', 15, 3, 2020): -0.10972,
 ('Eastern Finland', 16, 3, 2020): -0.07167000000000001,
 ('Eastern Finland', 17, 3, 2020): -0.13906,
 ('Eastern Finland', 18, 3, 2020): -0.15504,
 ('Eastern Finland', 19, 3, 2020): -0.18442,
 ('Eastern Finland', 20, 3, 2020): -0.20135,
 ('Eastern Finland', 21, 3, 2020): -0.1924,
 ('Eastern Finland', 22, 3, 2020)

# read Google mutipliers

In [24]:
# dfs = []
# filenames = ['/home/huangz4/nordicmathcovid-mobility/public_mobility_data/Google/2020_FI_Region_Mobility_Report.csv']
# for filename in filenames:
#     dfs.append(pd.read_csv(filename, parse_dates=['date']))
    
# df_google_fin = pd.concat(dfs)

# # Select the ones you want
# df_google_fin_selected_coloumns = df_google_fin[['sub_region_1','date','transit_stations_percent_change_from_baseline']]

# df_google_fin_trimmed= df_google_fin_selected_coloumns.dropna()

# df_google_fin_trimmed.to_csv('./public_mobility_data/google_multiplier.csv', index=False)

In [25]:
df_google_fin_trimmed = pd.read_csv('./public_mobility_data/google_multiplier.csv', 
                                    parse_dates=['date'])

In [26]:
google_multiplier_book = dict(zip(zip(df_google_fin_trimmed.date, df_google_fin_trimmed.sub_region_1), df_google_fin_trimmed.transit_stations_percent_change_from_baseline))
google_multiplier_book = {(key[0].to_pydatetime(), key[1]):value for (key, value) in google_multiplier_book.items()}
google_multiplier_book = {(key[1], key[0].day, key[0].month, key[0].year):(value/100) for (key, value) in google_multiplier_book.items()}
#change to relative change from baseline to make it in the same format as Facebook data

In [27]:
google_multiplier_book

{('Central Finland', 15, 2, 2020): 0.11,
 ('Central Finland', 16, 2, 2020): 0.11,
 ('Central Finland', 17, 2, 2020): -0.04,
 ('Central Finland', 18, 2, 2020): -0.07,
 ('Central Finland', 19, 2, 2020): 0.08,
 ('Central Finland', 20, 2, 2020): 0.06,
 ('Central Finland', 21, 2, 2020): 0.13,
 ('Central Finland', 22, 2, 2020): 0.33,
 ('Central Finland', 23, 2, 2020): 0.23,
 ('Central Finland', 24, 2, 2020): 0.05,
 ('Central Finland', 25, 2, 2020): -0.01,
 ('Central Finland', 26, 2, 2020): 0.1,
 ('Central Finland', 27, 2, 2020): 0.07,
 ('Central Finland', 28, 2, 2020): -0.09,
 ('Central Finland', 29, 2, 2020): 0.18,
 ('Central Finland', 1, 3, 2020): 0.1,
 ('Central Finland', 2, 3, 2020): -0.02,
 ('Central Finland', 3, 3, 2020): -0.08,
 ('Central Finland', 4, 3, 2020): 0.0,
 ('Central Finland', 5, 3, 2020): -0.05,
 ('Central Finland', 6, 3, 2020): -0.06,
 ('Central Finland', 7, 3, 2020): 0.05,
 ('Central Finland', 8, 3, 2020): 0.09,
 ('Central Finland', 9, 3, 2020): -0.09,
 ('Central Finland'

In [28]:
def make_aggregated_and_dynamic_estimations_from_facebook_multipliers(static_estimation_book, municipality_to_large_district_mapping, facebook_multiplier_book, m_to_f):
    set_of_unique_dates = {(tup[1], tup[2], tup[3]) for tup in facebook_multiplier_book.keys()}
    dynamic_estimation_municipality_level = {}
    dynamic_estimation_larger_region = {}
    
    
    not_found_set_1 = set()
    not_found_set_2 = set()
    problematic_municipalities = {'Brändö',  'Eckerö',  'Finström',  'Föglö',  'Geta',  'Hammarland',  'Jomala', \
                                  'Kumlinge',  'Kökar',  'Lemland',  'Lumparland',  'Mariehamn',  'Saltvik',  'Sottunga',  'Sund',  'Vårdö', 'Koski_Tl'}
    static_key_list = [tup for tup in list(static_estimation_book.keys()) if (tup[0] not in problematic_municipalities and tup[1] not in problematic_municipalities)]
    for key in static_key_list:
        static_estimation = static_estimation_book[key]
        origin_m = key[0]
        destination_m = key[1]
        origin_f = m_to_f[origin_m]
        destination_f = m_to_f[destination_m]
        #origin_large = municipality_to_large_district_mapping[origin_m]
        #destination_large = municipality_to_large_district_mapping[destination_m]
        for date_tup in list(set_of_unique_dates):
            d, m, y = date_tup[0], date_tup[1], date_tup[2]
            o_multiplier = facebook_multiplier_book[(origin_f, d, m, y)]
            d_multiplier = facebook_multiplier_book[(destination_f, d, m, y)]
            multiplier = (o_multiplier + d_multiplier)/2
            adjusted_estimation = static_estimation + static_estimation * multiplier
            # update dynamic_estimation_municipality_level
            dynamic_estimation_municipality_level[(origin_m, destination_m, '6', str(d), str(m), str(y))] = adjusted_estimation
            dynamic_estimation_municipality_level[(destination_m, origin_m, '18', str(d), str(m), str(y))] = adjusted_estimation
            
            # update dynamic_estimation_larger_region
            origin_large = municipality_to_large_district_mapping[origin_m]
            destination_large = municipality_to_large_district_mapping[destination_m]
            if (origin_large, destination_large, '6', str(d), str(m), str(y)) not in dynamic_estimation_larger_region.keys():
                dynamic_estimation_larger_region[(origin_large, destination_large, '6', str(d), str(m), str(y))] = 0
            dynamic_estimation_larger_region[(origin_large, destination_large, '6', str(d), str(m), str(y))] += adjusted_estimation
            if (destination_large, origin_large, '18', str(d), str(m), str(y)) not in dynamic_estimation_larger_region.keys():
                dynamic_estimation_larger_region[(destination_large, origin_large, '18', str(d), str(m), str(y))] = 0
            dynamic_estimation_larger_region[(destination_large, origin_large, '18', str(d), str(m), str(y))] += adjusted_estimation
    #print(len(not_found_set_2))
    #print(len(not_found_set_2))
    return dynamic_estimation_municipality_level, dynamic_estimation_larger_region

In [29]:
def make_aggregated_and_dynamic_estimations_from_google_multipliers(static_estimation_book, municipality_to_large_district_mapping, google_multiplier_book, m_to_g):
    set_of_unique_dates = {(tup[1], tup[2], tup[3]) for tup in google_multiplier_book.keys()}
    dynamic_estimation_municipality_level = {}
    dynamic_estimation_larger_region = {}
    
    
    not_found_set_1 = set()
    not_found_set_2 = set()
    problematic_municipalities = {'Brändö',  'Eckerö',  'Finström',  'Föglö',  'Geta',  'Hammarland',  'Jomala', \
                                  'Kumlinge',  'Kökar',  'Lemland',  'Lumparland',  'Mariehamn',  'Saltvik',  'Sottunga',  'Sund',  'Vårdö', 'Koski_Tl'}
    static_key_list = [tup for tup in list(static_estimation_book.keys()) if (tup[0] not in problematic_municipalities and tup[1] not in problematic_municipalities)]
    for key in static_key_list:
        static_estimation = static_estimation_book[key]
        origin_m = key[0]
        destination_m = key[1]
        origin_g = m_to_g[origin_m]
        destination_g = m_to_g[destination_m]
        #origin_large = municipality_to_large_district_mapping[origin_m]
        #destination_large = municipality_to_large_district_mapping[destination_m]
        for date_tup in list(set_of_unique_dates):
            d, m, y = date_tup[0], date_tup[1], date_tup[2]
            if (origin_g, d, m, y) in google_multiplier_book.keys():
                o_multiplier = google_multiplier_book[(origin_g, d, m, y)]
                if (destination_g, d, m, y) in google_multiplier_book.keys():
                    d_multiplier = google_multiplier_book[(destination_g, d, m, y)]
                    multiplier = (o_multiplier + d_multiplier)/2
                    adjusted_estimation = static_estimation + static_estimation * multiplier
                    # update dynamic_estimation_municipality_level
                    dynamic_estimation_municipality_level[(origin_m, destination_m, '6', str(d), str(m), str(y))] = adjusted_estimation
                    dynamic_estimation_municipality_level[(destination_m, origin_m, '18', str(d), str(m), str(y))] = adjusted_estimation
            
                    # update dynamic_estimation_larger_region
                    origin_large = municipality_to_large_district_mapping[origin_m]
                    destination_large = municipality_to_large_district_mapping[destination_m]
                    if (origin_large, destination_large, '6', str(d), str(m), str(y)) not in dynamic_estimation_larger_region.keys():
                        dynamic_estimation_larger_region[(origin_large, destination_large, '6', str(d), str(m), str(y))] = 0
                    dynamic_estimation_larger_region[(origin_large, destination_large, '6', str(d), str(m), str(y))] += adjusted_estimation
                    if (destination_large, origin_large, '18', str(d), str(m), str(y)) not in dynamic_estimation_larger_region.keys():
                        dynamic_estimation_larger_region[(destination_large, origin_large, '18', str(d), str(m), str(y))] = 0
                    dynamic_estimation_larger_region[(destination_large, origin_large, '18', str(d), str(m), str(y))] += adjusted_estimation
    #print(len(not_found_set_2))
    #print(len(not_found_set_2))
    return dynamic_estimation_municipality_level, dynamic_estimation_larger_region

# combine Google multipliers with radiation model

# Radiation + Google

In [30]:
dynamic_commuters_estimation_from_google_municipality_level, dynamic_commuters_estimation_from_google_larger_region = make_aggregated_and_dynamic_estimations_from_google_multipliers(commuters_static_radiation_municipality, m_to_h, google_multiplier_book, m_to_g)

In [31]:
len(dynamic_commuters_estimation_from_google_larger_region)

191372

# Gravity + Google

In [1]:
commuters_gravity_google_municipality_level, commuters_gravity_google_larger_region = make_aggregated_and_dynamic_estimations_from_google_multipliers(commuters_static_gravity_municipality, m_to_h, google_multiplier_book, m_to_g)

NameError: name 'make_aggregated_and_dynamic_estimations_from_google_multipliers' is not defined

In [ ]:
len(commuters_gravity_google_larger_region)

# Saving google estimations

# Saving Gravity + Google 

In [ ]:
import pickle
save_path = "./results/dynamic_commuters_gravity_google_hcd.pkl"
with open(save_path, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(commuters_gravity_google_larger_region, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
# saving radiation + Google

In [ ]:
import pickle
save_path = "./results/dynamic_commuters_radiation_google_hcd.pkl"
with open(save_path, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(dynamic_commuters_estimation_from_google_larger_region, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
separated_by_od = {}
for (origin, destination, h, d, m, y) in dynamic_commuters_estimation_from_google_larger_region.keys():
    if (origin, destination) not in separated_by_od.keys():
        separated_by_od[(origin, destination)] = {}
    separated_by_od[(origin, destination)][(h,d,m,y)] = dynamic_commuters_estimation_from_google_larger_region[(origin, destination, h, d, m, y)]

for (origin, destination) in separated_by_od.keys():
    dates = list(separated_by_od[(origin, destination)].keys())
    sorted_date_list = sorted(dates, key=lambda tup: (int(tup[3]),int(tup[2]),int(tup[1]),int(tup[0])))
    flow_list = [dynamic_commuters_estimation_from_google_larger_region[(origin, destination, h, d, m, y)] for (h,d,m,y) in sorted_date_list]
    print(origin)
    print(destination)
    plt.plot(range(len(sorted_date_list)), flow_list, ls = " ", marker = '.')
    plt.title(str(origin)+" to "+str(destination))
    plt.show()
#print(sorted_date_list)


# Facebook + radiation

In [ ]:
dynamic_commuters_estimation_from_facebook_municipality_level, dynamic_commuters_estimation_from_facebook_larger_region = make_aggregated_and_dynamic_estimations_from_facebook_multipliers(commuters_static_radiation_municipality, m_to_h, facebook_multiplier_book, m_to_f)

# Facebook + gravity

In [ ]:
commuters_facebook_gravity_municipality_level, commuters_facebook_gravity_larger_region = make_aggregated_and_dynamic_estimations_from_facebook_multipliers(commuters_static_gravity_municipality, m_to_h, facebook_multiplier_book, m_to_f)

In [ ]:
len(commuters_facebook_gravity_larger_region)

In [ ]:
import pickle
save_path = "./results/dynamic_commuters_gravity_facebook_hcd.pkl"
with open(save_path, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(commuters_facebook_gravity_larger_region, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
len(dynamic_commuters_estimation_from_facebook_larger_region)

In [ ]:
import pickle
save_path = "./results/dynamic_commuters_radiation_facebook_hcd.pkl"
with open(save_path, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(dynamic_commuters_estimation_from_facebook_larger_region, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
separated_by_od = {}
for (origin, destination, h, d, m, y) in dynamic_commuters_estimation_from_facebook_larger_region.keys():
    if (origin, destination) not in separated_by_od.keys():
        separated_by_od[(origin, destination)] = {}
    separated_by_od[(origin, destination)][(h,d,m,y)] = dynamic_commuters_estimation_from_facebook_larger_region[(origin, destination, h, d, m, y)]

for (origin, destination) in separated_by_od.keys():
    dates = list(separated_by_od[(origin, destination)].keys())
    sorted_date_list = sorted(dates, key=lambda tup: (int(tup[3]),int(tup[2]),int(tup[1]),int(tup[0])))
    flow_list = [dynamic_commuters_estimation_from_facebook_larger_region[(origin, destination, h, d, m, y)] for (h,d,m,y) in sorted_date_list]
    print(origin)
    print(destination)
    plt.plot(range(len(sorted_date_list)), flow_list, ls = " ", marker = '.')
    plt.title(str(origin)+" to "+str(destination))
    plt.show()
#print(sorted_date_list)
